In [1]:
import sys
import time
import keras
import numpy as np
import keras.backend as K
import matplotlib.pyplot as plt

from scipy import spatial
from keras.layers import *
from keras import regularizers
from keras.utils import np_utils
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.convolutional import *    
from keras.layers.core import Dense, Activation, Dropout
from keras.optimizers import SGD, Adam
from keras.callbacks import Callback as Callback
from keras.callbacks import ModelCheckpoint, EarlyStopping

sys.path.append("../../../")

%run ../../../ds/unity/npz/generator.py

K.set_image_dim_ordering('tf')
%matplotlib inline

Using TensorFlow backend.


# prepare dataset

In [2]:
NPZ_PATH = "/home/chy/archive-data/processed/unity-npz"
CHK_POINT_PATH = "./unity-weights.{epoch:02d}-{val_loss:.7f}.hdf5"

# callbacks

In [3]:
history = TimeHistory()
checkpoint = 

stopper = EarlyStopping(monitor='val_loss', 
                        min_delta=0, 
                        patience=5, 
                        verbose=0, 
                        mode='auto', 
                        baseline=None, 
                        restore_best_weights=True)

callbacks = [history, checkpointer, stopper]

# Model

In [4]:
INPUT_SHAPE = (80, 80, 1)

sam = Sequential()
sam.add(Conv2D(64, kernel_size=(1,1), padding='same', input_shape=INPUT_SHAPE))
sam.add(LeakyReLU())
sam.add(Conv2D(64, kernel_size=(3,3), padding='same'))
sam.add(LeakyReLU())
sam.add(Conv2D(128, kernel_size=(3,3), padding='same'))
sam.add(LeakyReLU())
sam.add(MaxPooling2D(pool_size=(2,2), padding='same'))

sam.add(Conv2D(128, kernel_size=(3,3), padding='same'))
sam.add(LeakyReLU())
sam.add(Conv2D(192, kernel_size=(3,3), padding='same'))
sam.add(LeakyReLU())
sam.add(MaxPooling2D(pool_size=(2,2)))

sam.add(Conv2D(192, kernel_size=(3,3), padding='same'))
sam.add(LeakyReLU())
sam.add(MaxPooling2D(pool_size=(2,2)))

sam.add(Flatten())
sam.add(Dense(1600))
sam.add(LeakyReLU())
sam.add(Dense(512))
sam.add(LeakyReLU())
sam.add(Dense(3))

sam.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 80, 80, 64)        128       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 80, 80, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 80, 80, 64)        36928     
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 80, 80, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 80, 80, 128)       73856     
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 80, 80, 128)       0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 40, 40, 128)       0         
__________

# Losses

In [5]:
def cosine_dist(y_true, y_pred):
    y_true = y_true
    y_pred = y_pred
    y_true = K.l2_normalize(y_true, axis=-1)
    y_pred = K.l2_normalize(y_pred, axis=-1)
    return K.mean(1 - K.sum((y_true * y_pred), axis=-1))

sam.compile(loss=losses.sparse_categorical_crossentropy, 
            optimizer='adam', 
            metrics=['accuracy'])

# hyper-params

In [6]:
EXP_CODE = "home1"
NUM_EPOCH = 40
BATCH_SIZE = 256

In [7]:
model_json = sam.to_json()
with open("./unity-model-exp-{}.json".format(EXP_CODE), "w") as json_file : 
    json_file.write(model_json)

In [8]:
gen_train = UnityGenerator(NPZ_PATH, batch_size=BATCH_SIZE, purpose="train")
gen_valid = UnityGenerator(NPZ_PATH, batch_size=BATCH_SIZE, purpose="valid")

In [9]:
sam.fit_generator(generator=gen_train, 
                  validation_data=gen_valid,
                  callbacks=callbacks,
                  epochs=NUM_EPOCH,
                  use_multiprocessing=True, 
                  workers=2)

Epoch 1/40
2464/2464 [==============================] - 4513s 2s/step - loss: 0.0163 - acc: 0.9440 - cosine_dist: 0.0163 - val_loss: 3.8279e-04 - val_acc: 0.9867 - val_cosine_dist: 3.8279e-04
Epoch 2/40
2464/2464 [==============================] - 4306s 2s/step - loss: 3.1145e-04 - acc: 0.9889 - cosine_dist: 3.1145e-04 - val_loss: 2.1213e-04 - val_acc: 0.9909 - val_cosine_dist: 2.1213e-04
Epoch 3/40
2464/2464 [==============================] - 4246s 2s/step - loss: 2.2226e-04 - acc: 0.9913 - cosine_dist: 2.2226e-04 - val_loss: 1.7803e-04 - val_acc: 0.9925 - val_cosine_dist: 1.7803e-04
Epoch 4/40
2464/2464 [==============================] - 4234s 2s/step - loss: 1.8999e-04 - acc: 0.9918 - cosine_dist: 1.8999e-04 - val_loss: 1.5520e-04 - val_acc: 0.9928 - val_cosine_dist: 1.5520e-04
Epoch 5/40
2464/2464 [==============================] - 4237s 2s/step - loss: 1.2880e-04 - acc: 0.9930 - cosine_dist: 1.2880e-04 - val_loss: 1.2175e-04 - val_acc: 0.9935 - val_cosine_dist: 1.2175e-04
Epoch 6/

Process ForkPoolWorker-63:
Process ForkPoolWorker-64:
Process ForkPoolWorker-61:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/intel/intelpython3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/opt/intel/intelpython3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/opt/intel/intelpython3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/opt/intel/intelpython3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/intel/intelpython3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/intel/intelpython3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/intel/intelpython3/lib/python3.6/multiprocessing

KeyboardInterrupt: 

In [ ]:
# model_path = "/home/chy/archive-model/incubator/ove/exp1-unity-model.json"
# weight_path = "/home/chy/archive-model/incubator/ove/exp1-unity-weights.hdf5"

# with open(model_path, "r") as json_file:
#     model_json = json_file.read() 
#     model = model_from_json(model_json)

# model.load_weights(weight_path)